## DuckDB Notebook

This notebook generates a bunch of raw outputs, without applying PAC, to be consumed by a second stage.

```
 group by              output cols                        
 key cols ┌────────┬────────┬────────┬────────┐           
        │ │   A    │   B    │   C    │   D    │           
      ┌─▼─┼────────┼────────┼────────┼────────┤           
      │ 1 │   2    │        │        │        │           
      ├───┼───|────┼────────┼────────┼────────┤           
      │ 2 │   │    │        │        │        │           
      ├───┼───┼────┼────────┼────────┼────────┤           
      │ 3 │   │    │        │        │        │           
      └───┴───┼────┴────────┴────────┴────────┘           
              ▼                 A_1.json                  
       Sample 0:   A1=2        ┌─────────────────────────┐
       Sample 1:   A1=4  ───▶  │{                        │
             ...               │    col: A               │
       Sample 999: A1=3        │    row: 1               │
                               │    value: [2, 4, ... 3] │
                               │}                        │
                               └─────────────────────────┘
```

In [1]:
#!/usr/bin/env python
# coding: utf-8

import os
import pickle
import shutil

import duckdb
import polars as pl

In [2]:
EXPERIMENT = "pac-duckdb-q1"
OUTPUT_DIR = f"./outputs/{EXPERIMENT}-step1"
SAMPLES = 1024

SAMPLE_STEP = f"""
DROP TABLE IF EXISTS random_samples;

CREATE TABLE random_samples AS
WITH sample_numbers AS MATERIALIZED (
    SELECT range AS sample_id FROM range({SAMPLES//2})
), random_values AS MATERIALIZED (
    SELECT 
        sample_numbers.sample_id,
        customer.rowid AS row_id,
        (RANDOM() > 0.5)::BOOLEAN AS random_binary
    FROM sample_numbers
    JOIN customer ON TRUE  -- Cross join to duplicate rows for each sample
)
SELECT
    sample_id,
    row_id,
    random_binary
FROM random_values
UNION ALL
SELECT -- select the complementary samples too
    ({SAMPLES//2}) + sample_id,
    row_id,
    NOT random_binary  -- Inverse the random_binary to get the complementary sample
FROM random_values
ORDER BY sample_id, row_id;
"""

PREPARE_STEP = """
DEALLOCATE PREPARE run_query;

PREPARE run_query AS 
SELECT
    l_returnflag,
    l_linestatus,
    2*sum(l_quantity) AS sum_qty,
    2*sum(l_extendedprice) AS sum_base_price,
    2*sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,
    2*sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,
    avg(l_quantity) AS avg_qty,
    avg(l_extendedprice) AS avg_price,
    avg(l_discount) AS avg_disc,
    2*count(*) AS count_order
FROM
    lineitem
JOIN orders ON lineitem.l_orderkey = orders.o_orderkey
JOIN customer ON orders.o_custkey = customer.c_custkey
JOIN random_samples AS rs
    ON rs.row_id = customer.rowid
WHERE
    l_shipdate <= CAST('1998-09-02' AS date)
    AND rs.random_binary = TRUE
    AND rs.sample_id = $sample
GROUP BY
    l_returnflag,
    l_linestatus
ORDER BY
    l_returnflag,
    l_linestatus;
"""

INDEX_COLS = ['l_returnflag', 'l_linestatus']
OUTPUT_COLS = ['sum_qty', 'sum_base_price', 'sum_disc_price', 'sum_charge', 'avg_qty', 'avg_price', 'avg_disc', 'count_order']

In [3]:
# Parameters
EXPERIMENT = "autopac-duckdb-q1"
OUTPUT_DIR = "./outputs/autopac-duckdb-q1-step1"
SAMPLES = 1024
QUERY_STEP = "DROP TABLE IF EXISTS random_samples;\n\nCREATE TABLE random_samples AS\nWITH sample_numbers AS MATERIALIZED (\n    SELECT range AS sample_id FROM range(512)\n), random_values AS MATERIALIZED (\n    SELECT \n        sample_numbers.sample_id,\n        customer.rowid AS row_id,\n        (RANDOM() > 0.5)::BOOLEAN AS random_binary\n    FROM sample_numbers\n    JOIN customer ON TRUE  -- Cross join to duplicate rows for each sample\n)\nSELECT\n    sample_id,\n    row_id,\n    random_binary\nFROM random_values\nUNION ALL\nSELECT -- select the complementary samples too\n    (512) + sample_id,\n    row_id,\n    NOT random_binary  -- Inverse the random_binary to get the complementary sample\nFROM random_values\nORDER BY sample_id, row_id;"
PREPARE_STEP = "DEALLOCATE PREPARE run_query;\n\nPREPARE run_query AS \nSELECT\n    l_returnflag,\n    l_linestatus,\n    2*sum(l_quantity) AS sum_qty,\n    2*sum(l_extendedprice) AS sum_base_price,\n    2*sum(l_extendedprice * (1 - l_discount)) AS sum_disc_price,\n    2*sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) AS sum_charge,\n    avg(l_quantity) AS avg_qty,\n    avg(l_extendedprice) AS avg_price,\n    avg(l_discount) AS avg_disc,\n    2*count(*) AS count_order\nFROM\n    lineitem\nJOIN orders ON lineitem.l_orderkey = orders.o_orderkey\nJOIN customer ON orders.o_custkey = customer.c_custkey\nJOIN random_samples AS rs\n    ON rs.row_id = customer.rowid\nWHERE\n    l_shipdate <= CAST('1998-09-02' AS date)\n    AND rs.random_binary = TRUE\n    AND rs.sample_id = $sample\nGROUP BY\n    l_returnflag,\n    l_linestatus\nORDER BY\n    l_returnflag,\n    l_linestatus;"
INDEX_COLS = ["l_returnflag", "l_linestatus"]
OUTPUT_COLS = ["sum_qty", "sum_base_price", "sum_disc_price", "sum_charge", "avg_qty", "avg_price", "avg_disc", "count_order"]


In [4]:
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [5]:
# duckdb load data/tpch/tpch.duckdb into the temporary in-memory database
con = duckdb.connect(database=':memory:')
tables = ["customer", "lineitem", "nation", "orders", "part", "partsupp", "region", "supplier"]
for t in tables:
    con.execute(f"CREATE TABLE {t} AS SELECT * FROM 'data/tpch/{t}.parquet'")

In [6]:
# Construct the table of random samples
# to use, join it with the lineitem table (for specific sample # s) and filter to just the
# rows where random_binary = 1.0
# This will give us a 50% sample of the lineitem table for each sample # s

assert SAMPLES % 2 == 0, "SAMPLES must be even to create complementary samples."

random_samples = con.execute(SAMPLE_STEP)

The randomness of what rows are chosen is saved to disk in `random_binary.json`. For each sample #, there is an array with one entry per row, where 1 means the row was chosen and 0 means it was not.

In [7]:
con.execute("""
SELECT sample_id, array_agg(random_binary::TINYINT) as random_binary
FROM random_samples
GROUP BY sample_id;
""").pl().write_json(f"{OUTPUT_DIR}/random_binary.json")

Query is specified as a prepared statement. We will then execute it once per sample.

In [8]:
# Query
con.execute(PREPARE_STEP)

# Run query to see output
dfs0 = con.execute(f"EXECUTE run_query(sample := {0});").pl()

# Save csv copies of the first 5 samples
os.makedirs(f"{OUTPUT_DIR}/csv", exist_ok=True)
for s in range(5):
    con.execute(f"EXECUTE run_query(sample := {s});").pl().write_csv(f"{OUTPUT_DIR}/csv/sample_{s}.csv")

dfs0

shape: (4, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ l_returnf ┆ l_linesta ┆ sum_qty   ┆ sum_base_ ┆ … ┆ avg_qty   ┆ avg_price ┆ avg_disc ┆ count_ord │
│ lag       ┆ tus       ┆ ---       ┆ price     ┆   ┆ ---       ┆ ---       ┆ ---      ┆ er        │
│ ---       ┆ ---       ┆ decimal[3 ┆ ---       ┆   ┆ f64       ┆ f64       ┆ f64      ┆ ---       │
│ str       ┆ str       ┆ 8,2]      ┆ decimal[3 ┆   ┆           ┆           ┆          ┆ i64       │
│           ┆           ┆           ┆ 8,2]      ┆   ┆           ┆           ┆          ┆           │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ A         ┆ F         ┆ 3779374.0 ┆ 533282817 ┆ … ┆ 25.557378 ┆ 36062.349 ┆ 0.049977 ┆ 147878    │
│           ┆           ┆ 0         ┆ 9.82      ┆   ┆           ┆ 909       ┆          ┆           │
│ N         ┆ F         ┆ 96208.00  ┆ 134862482 ┆ … ┆ 25.519363 ┆ 35772.541 ┆ 0.048589 ┆ 3770      │
│           ┆           ┆           ┆ .70       ┆   ┆           ┆ 83        ┆          ┆           │
│ N         ┆ O         ┆ 7457608.0 ┆ 105088023 ┆ … ┆ 25.520351 ┆ 35961.708 ┆ 0.050082 ┆ 292222    │
│           ┆           ┆ 0         ┆ 29.56     ┆   ┆           ┆ 323       ┆          ┆           │
│ R         ┆ F         ┆ 3785848.0 ┆ 533624916 ┆ … ┆ 25.563472 ┆ 36032.365 ┆ 0.050013 ┆ 148096    │
│           ┆           ┆ 0         ┆ 3.96      ┆   ┆           ┆ 249       ┆          ┆           │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴──────────┴───────────┘

In [9]:
# Run the query for each sample, but accumulate in a pl.DataFrame instead of a list
dfsdf: pl.DataFrame = pl.concat(
    con.execute(f"EXECUTE run_query(sample := {s});").pl().insert_column(0, pl.lit(s).alias("sample"))
    for s in range(SAMPLES)
)
dfsdf

shape: (4_096, 11)
┌────────┬────────────┬────────────┬────────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ sample ┆ l_returnfl ┆ l_linestat ┆ sum_qty    ┆ … ┆ avg_qty   ┆ avg_price ┆ avg_disc ┆ count_ord │
│ ---    ┆ ag         ┆ us         ┆ ---        ┆   ┆ ---       ┆ ---       ┆ ---      ┆ er        │
│ i32    ┆ ---        ┆ ---        ┆ decimal[38 ┆   ┆ f64       ┆ f64       ┆ f64      ┆ ---       │
│        ┆ str        ┆ str        ┆ ,2]        ┆   ┆           ┆           ┆          ┆ i64       │
╞════════╪════════════╪════════════╪════════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 0      ┆ A          ┆ F          ┆ 3779374.00 ┆ … ┆ 25.557378 ┆ 36062.349 ┆ 0.049977 ┆ 147878    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 909       ┆          ┆           │
│ 0      ┆ N          ┆ F          ┆ 96208.00   ┆ … ┆ 25.519363 ┆ 35772.541 ┆ 0.048589 ┆ 3770      │
│        ┆            ┆            ┆            ┆   ┆           ┆ 83        ┆          ┆           │
│ 0      ┆ N          ┆ O          ┆ 7457608.00 ┆ … ┆ 25.520351 ┆ 35961.708 ┆ 0.050082 ┆ 292222    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 323       ┆          ┆           │
│ 0      ┆ R          ┆ F          ┆ 3785848.00 ┆ … ┆ 25.563472 ┆ 36032.365 ┆ 0.050013 ┆ 148096    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 249       ┆          ┆           │
│ 1      ┆ A          ┆ F          ┆ 3752128.00 ┆ … ┆ 25.569557 ┆ 36034.090 ┆ 0.050246 ┆ 146742    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 107       ┆          ┆           │
│ …      ┆ …          ┆ …          ┆ …          ┆ … ┆ …         ┆ …         ┆ …        ┆ …         │
│ 1022   ┆ R          ┆ F          ┆ 3764552.00 ┆ … ┆ 25.514775 ┆ 35980.585 ┆ 0.050007 ┆ 147544    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 995       ┆          ┆           │
│ 1023   ┆ A          ┆ F          ┆ 3787522.00 ┆ … ┆ 25.513789 ┆ 35982.591 ┆ 0.050065 ┆ 148450    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 012       ┆          ┆           │
│ 1023   ┆ N          ┆ F          ┆ 96500.00   ┆ … ┆ 25.461741 ┆ 35574.086 ┆ 0.049361 ┆ 3790      │
│        ┆            ┆            ┆            ┆   ┆           ┆ 734       ┆          ┆           │
│ 1023   ┆ N          ┆ O          ┆ 7499070.00 ┆ … ┆ 25.514157 ┆ 35947.716 ┆ 0.050168 ┆ 293918    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 908       ┆          ┆           │
│ 1023   ┆ R          ┆ F          ┆ 3810782.00 ┆ … ┆ 25.520224 ┆ 36018.103 ┆ 0.050088 ┆ 149324    │
│        ┆            ┆            ┆            ┆   ┆           ┆ 882       ┆          ┆           │
└────────┴────────────┴────────────┴────────────┴───┴───────────┴───────────┴──────────┴───────────┘

In [10]:
# Define which columns are the group-by keys (INDEX_COLS) and which are the output columns (OUTPUT_COLS)
# - moved to parameters cell at top of notebook

# Save these to disk for later use
with open(f"{OUTPUT_DIR}/INDEX_COLS.pkl", 'wb') as f:
    pickle.dump(INDEX_COLS, f)
with open(f"{OUTPUT_DIR}/OUTPUT_COLS.pkl", 'wb') as f:
    pickle.dump(OUTPUT_COLS, f)

In [11]:
# Combine all the samples into one table, grouped-by the group-by keys. Each cell contains an n <= # of samples length array of values.
listdf = (dfsdf.group_by(INDEX_COLS, maintain_order=True)
      .all()
      .drop("sample"))
listdf

shape: (4, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ l_returnf ┆ l_linesta ┆ sum_qty   ┆ sum_base_ ┆ … ┆ avg_qty   ┆ avg_price ┆ avg_disc  ┆ count_or │
│ lag       ┆ tus       ┆ ---       ┆ price     ┆   ┆ ---       ┆ ---       ┆ ---       ┆ der      │
│ ---       ┆ ---       ┆ list[deci ┆ ---       ┆   ┆ list[f64] ┆ list[f64] ┆ list[f64] ┆ ---      │
│ str       ┆ str       ┆ mal[38,2] ┆ list[deci ┆   ┆           ┆           ┆           ┆ list[i64 │
│           ┆           ┆ ]         ┆ mal[38,2] ┆   ┆           ┆           ┆           ┆ ]        │
│           ┆           ┆           ┆ ]         ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ A         ┆ F         ┆ [3779374. ┆ [53328281 ┆ … ┆ [25.55737 ┆ [36062.34 ┆ [0.049977 ┆ [147878, │
│           ┆           ┆ 00, 37521 ┆ 79.82,    ┆   ┆ 8, 25.569 ┆ 9909, 360 ┆ ,         ┆ 146742,  │
│           ┆           ┆ 28.00, …  ┆ 528771445 ┆   ┆ 557, …    ┆ 34.090107 ┆ 0.050246, ┆ …        │
│           ┆           ┆ 378…      ┆ 0.54,…    ┆   ┆ 25.51…    ┆ , ……      ┆ …         ┆ 148450]  │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ 0.05006…  ┆          │
│ N         ┆ F         ┆ [96208.00 ┆ [13486248 ┆ … ┆ [25.51936 ┆ [35772.54 ┆ [0.048589 ┆ [3770,   │
│           ┆           ┆ ,         ┆ 2.70, 131 ┆   ┆ 3, 25.316 ┆ 183, 3577 ┆ ,         ┆ 3680, …  │
│           ┆           ┆ 93166.00, ┆ 641499.88 ┆   ┆ 848, …    ┆ 2.146707, ┆ 0.050005, ┆ 3790]    │
│           ┆           ┆ …         ┆ , ……      ┆   ┆ 25.46…    ┆ … …       ┆ …         ┆          │
│           ┆           ┆ 96500.0…  ┆           ┆   ┆           ┆           ┆ 0.04936…  ┆          │
│ N         ┆ O         ┆ [7457608. ┆ [10508802 ┆ … ┆ [25.52035 ┆ [35961.70 ┆ [0.050082 ┆ [292222, │
│           ┆           ┆ 00, 73737 ┆ 329.56,   ┆   ┆ 1, 25.538 ┆ 8323, 360 ┆ ,         ┆ 288726,  │
│           ┆           ┆ 60.00, …  ┆ 103942902 ┆   ┆ 954, …    ┆ 00.534089 ┆ 0.050088, ┆ …        │
│           ┆           ┆ 749…      ┆ 05.2…     ┆   ┆ 25.51…    ┆ , ……      ┆ …         ┆ 293918]  │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ 0.05016…  ┆          │
│ R         ┆ F         ┆ [3785848. ┆ [53362491 ┆ … ┆ [25.56347 ┆ [36032.36 ┆ [0.050013 ┆ [148096, │
│           ┆           ┆ 00, 37499 ┆ 63.96,    ┆   ┆ 2, 25.573 ┆ 5249, 360 ┆ ,         ┆ 146630,  │
│           ┆           ┆ 08.00, …  ┆ 528601551 ┆   ┆ 948, …    ┆ 50.027349 ┆ 0.050071, ┆ …        │
│           ┆           ┆ 381…      ┆ 0.18,…    ┆   ┆ 25.52…    ┆ , ……      ┆ …         ┆ 149324]  │
│           ┆           ┆           ┆           ┆   ┆           ┆           ┆ 0.05008…  ┆          │
└───────────┴───────────┴───────────┴───────────┴───┴───────────┴───────────┴───────────┴──────────┘

In [12]:
# What are all the possible group-by key combinations?
allgroups: pl.DataFrame = listdf.select(INDEX_COLS)
allgroups.to_dicts()

[{'l_returnflag': 'A', 'l_linestatus': 'F'},
 {'l_returnflag': 'N', 'l_linestatus': 'F'},
 {'l_returnflag': 'N', 'l_linestatus': 'O'},
 {'l_returnflag': 'R', 'l_linestatus': 'F'}]

In [13]:
# Template for the final output, including all possible group-by groups
# Obtained by collecting all the samples in a big table and then keeping only the first occurrence of each groupby key.
# Then, fill all OUTPUT_COLS with nulls
templatedf = dfsdf.drop("sample").group_by(INDEX_COLS, maintain_order=True).first()
templatedf = templatedf.clear(n=len(allgroups)).with_columns(allgroups)
templatedf

with open(f"{OUTPUT_DIR}/template.pkl", "wb") as f:
    pickle.dump(templatedf, f)

In [14]:
# Write all table entries in the output table to their own JSON files. Each file has a number, the information of which file corresponds to which table entry
# is stored in reverse_map.json (as well as in the files themselves)
os.makedirs(f"{OUTPUT_DIR}/json", exist_ok=True)
os.makedirs(f"{OUTPUT_DIR}/pkl", exist_ok=True)
i: int = 0
for col in OUTPUT_COLS:
    for group in allgroups.iter_rows(named=True):
        values = listdf.filter(pl.col(k).eq(v) for k, v in group.items()).select(col).to_series()
        j = pl.DataFrame().with_columns([
            pl.lit(col).alias("col"),
            pl.lit(group).alias("row"),
            pl.lit(values.explode().dtype.__repr__()).alias("dtype"),
            values.alias("values"),
        ])
        j.write_json(f"{OUTPUT_DIR}/json/{i}.json")
        i+=1

In [ ]:
# zip the OUTPUT_DIR
shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)